In [1]:
import pandas as pd
import re
import json
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from google.colab import files

# These were needed to use anything in nltk on google colab
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
reviews = pd.read_pickle('/content/gdrive/MyDrive/reviews.pkl')

In [4]:
# processes each individual review and returns it to preprocessList
def preprocessString(text):
    text = text.lower()

    # Remove Urls
    urls = re.compile(re.compile(r'(https?:\/\/[www\.]?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,6}\b[-a-zA-Z0-9@:%_\+.~#?&//=]*)'))
    text = re.sub(urls, '', text)

    # Remove Hashtags
    hashtags = re.compile(r'\B#\w*[a-zA-Z]+\w*')
    text = re.sub(hashtags, '',text)

    # Remove @ signs
    atSigns = re.compile(r'\B@\w*[a-z0-9]+\w*')
    text = re.sub(atSigns, '', text)

    # Remove Emojis
    emojis = re.compile(r'[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text = re.sub(emojis, '', text)

    # Remove Punctuations
    punctuations = re.compile(r'\W+')
    text = re.sub(punctuations, ' ', text)
    
    # Remove Digits
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    token = nltk.word_tokenize(text)

    # Remove stopwords
    stopWords = set(nltk.corpus.stopwords.words('english'))
    filteredToken = []
    for t in token:
      if t not in stopWords:
        filteredToken.append(t)

    # Lemmatize the review and return the list to preprocessList
    lemmatize = [WordNetLemmatizer().lemmatize(x) for x in filteredToken]
    lemmatize = [x for x in lemmatize if x not in['rt','http'] and len(x)>1]
    return lemmatize

In [5]:
# Takes a list of different reviews and processes each and adds it to a new list to be returned. 
def preprocessList(lst):
  processedList = []
  if lst:
    for i in range(len(lst)):
      if lst[i]['text']:
        processedList.append(preprocessString(lst[i]['text']))
    if preprocessList:
      return processedList
    else:
      return None

In [6]:
# Save the processed data under a new row named processedReviews
reviews.loc['processedReviews'] = reviews.apply(lambda col: preprocessList(col['reviews']), axis = 0)

In [7]:
# Save the dataframe to a .pkl file and download it to your computer. 
reviews.to_pickle('reviews_processed.pkl')
files.download('reviews_processed.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
reviews

,B079YF1K1B,B08XYDDXDC,B08LF5G6J1,B0013OXFTU,B0013OXA90,B07HWMCWRC,B0036FWOJQ,B0058BM5AE,B000FGZG4O,B08MWVSKST,B00331VVGK,B001680S1G,B08437WBN9,B07VCGZJCL,B077V31X84,B000GFHPJI,B00012NCZW,B01I1YS9DG,B07QGP7PTD,B00UP9T9JY,B00020HQ00,B00IB14Q46,B00JVYRITC,B086CJD8L2,B0017OB05M,B00JUZMZT0,B000FH1138,B0013OXA8G,B01N26G89C,B08NRLJ7TM,B07WWB12VT,B0013OUQW4,B00B0X8B48,B07DLLGRJB,B000OYMJB0,B00HETCGES,B0898P5C55,B0053Y0ZB2,B0195J8GXE,B07PFYH2PQ,B07QD2GTCT,B0025Q1LNW,B07F97TF2X,B000UZVT5K,B004R63UY6,B0931SRN49,B07Z6VFMM4,B00S1XUS1Y,B00028LTAU,B0083USDVM,B01MT69YVC,B000I4FC1A,B007A506ES,B072JG37CY,B01EYKJ2LO,B07F97PHXY,B00068S90Y,B07F2KWN6C,B08WBNP49W,B08TZJRKX5,B07CGZ6DWF,B00CKU4OVO,B083V6MTYM,B082V5T56F,B07YVJDD2Q,B08KHQD7MX,B005P0TQ14,B096MSXY4B,B0739KKHWL,B0746QHQHJ,B08YPBTVFL,B0747S5Y9V,B07F91122K,B077QGJ6YV,B0001VW5N8,B00LC7A4KG,B00309ZDHS,B0183BYQW0,B08B19ZL4W
count,120,120,90,120,60,200,120,60,60,120,60,60,120,180,90,60,90,100,90,90,60,120,60,50,30,None,60,120,50,120,90,60,100,120,60,60,None,60,30,210,30,90,30,60,60,None,75,60,60,None,None,60,None,60,None,None,None,None,50,100,180,None,None,60,60,None,None,None,90,90,180,200,None,None,60,None,60,100,None
name,"PhosphatidylSerine 300mg Per Serving, Made in ...","Nutricost Phosphatidylserine 400mg, 120 Capsul...",Ultra High Purity Phosphatidylserine from Ital...,"NOW Supplemnets, Phosphatidyl Serine 100 mg wi...",Jarrow Formulas PS 100-60 Softgels - 100 mg Ph...,Phosphatidylserine Memory Ultimate Complex 100...,"Doctor's Best Best Phosphatidyl Serine 100, 12...",Integrative Therapeutics - Phosphatidylserine ...,Thorne Research - Iso-Phos - Phosphatidylserin...,PhosphatidylSerine & Bacopa Monnieri 400 mg 2 ...,"Solgar Phosphatidyl-Serine 200 mg, 60 Softgels...",Pure Encapsulations PS 100 | Phosphatidylserin...,"PhosphatidylSerine & Bacopa Monnieri, Better T...",Phosphatidylserine 500mg 180 Vegetarian Capsul...,"Phosphatidylserine 100mg, 100% Soy Free, Best ...",Source Naturals Phosphatidyl Serine Complex St...,Phosphatidylserine Complex 500mg Softgel – Sup...,Phosphatidylserine Memory Ultimate 1000mg (per...,PhosphatidylSerine 200mg Complex - 90 Vegan Ca...,Soy Free Phosphatidylserine 100mg - Phosphatid...,"Nature's Way Phosphatidylserine, 100 mg per se...","Natural Factors, Phosphatidylserine, Support f...",Klaire Labs Phosphatidyl Serine sf - Phosphati...,Neuriva Nootropic Brain Support Supplement - O...,Swanson Phosphatidylserine Memory Brain and Co...,Klaire Labs Phosphatidyl Serine - 500 Milligra...,Designs for Health PS 150 Phosphatidylserine C...,Jarrow Formulas PS 100 - 120 Capsules - 100 mg...,"NOW Supplements, Phosphatidyl Serine 300 mg, E...","18,000mg 10x Concentrated Ultra High Strength ...",ONNIT Alpha Brain (180ct) - Premium Nootropic ...,"NOW Supplements, Phosphatidyl Serine 100 mg wi...","Life Extension Phosphatidylserine 100 mg, 100 ...",Healthy Origins Sunflower PS (Phosphatidylseri...,Country Life Vitamins Phosphatidylserine Compl...,Phosphatidyl Serine | Provides 100 mg of Phoso...,Metagenics Phosphatidylserine - Helps Support ...,"NOW Supplements, Phosphatidyl Serine 150 mg wi...",True-PS Soy Free Stabilized Phosphatidylserine...,"Phosphatidylcholine 1,200mg – 210 Softgels – E...",Cortisol Manager Allergen Free - Integrative T...,"BlueBonnet Power Thought Supplement, 90 Count",Swanson Triple-Strength Phosphatidylserine 300...,Olympian Labs Phosphatidylserine Complex | 60 ...,"Puritans Pride Neuro-ps 300 Mg Softgels, 60 Count",PhosphatidylSerine 100mg for Focus & Memory - ...,Neuriva Nootropic Brain Support Supplement - O...,"LES Labs Cortisol Health, Adrenal Support Supp...",PS (PhosphatidylSerine) 100mg (from 500mg Mixe...,Protocol For Life Balance - Phosphatidyl Serin...,"NaturesPlus Pedi-Active LECI-PS, DMAE Complex ...","Solaray Phosphatidylserine Plus Supplement, 10...",NuTru P-Tidyl-Wei Vegan Herbal Supplement: 150...,"Havasu Nutrition Ginkgo Biloba Non-GMO 120mg, ...","Solaray Sunflower Phosphatidylserine Non GMO, ...",Swanson Phosphatidylserine 100 Mil